In [2]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

# Gesell

Adapted from [unit 9: Gesell.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/Gesell.odc).


Associated lecture video: Unit 9 Lesson 6

## Problem statement

Gesell Developmental Schedules  measure  child development. The scoring is devised by the American child psychologist and pediatrician Arnold Gesell who founded the Clinic of Child Development at Yale in 1911 and directed it for many years.  The Gesell Developmental Schedules are a gauge of the status of a child's motor and language development and personal-social and adaptive behaviors.

For 21 children the age in months at which they first spoke and their Gesell Adaptive Score, which is the result of an aptitude test taken much later was recorded.  These data were originally collected by L. M. Linde of UCLA but were first published by M. R. Mickey, O. J. Dunn, and V. Clark, †Note on the use of stepwise regression in detecting outliers,† Computers and Biomedical Research, 1 (1967), pp. 105†111. 

The data have been used by several authors, e.g.,  N. R. Draper and J. A. John, †Influential observations and outliers in regression,† Technometrics, 23 (1981), pp. 21†26.

Let X be the age in in months a child speaks his/her first word and let Y be the Gesell adaptive score, a measure of a child†s aptitude (observed later on). 

Correlation of r = ¢0.640 indicates a  moderately strong negative relationship between age at first word spoken and Gesell score. How does the child†s aptitude change with how long it takes them to speak?

Are there any influential observations/potential outliers?

The solution follows a two-stage model:

a. Obtain mean ICPOs. Outside BUGS take reciprocal of mean ICPOs (these are CPOs) and consder them as an input for model B.

b. Read in and calibrate CPOs with posterior ordinates.

In [3]:
# fmt: off
x = np.array(
    (15, 26, 10, 9, 15, 20, 18, 11, 8, 20, 7, 9, 10, 11, 11, 10, 12, 42, 17, 11, 10)
)

y = np.array(
    (95, 71, 83, 91, 102, 87, 93, 100, 104, 94, 113, 96, 83, 84, 102, 100, 105, 57, 121, 86, 100)
)
# fmt: on

## Part A

In [7]:
with pm.Model() as model_a:
    # couldn't get .001 working, answers are pretty close with .003
    tau = pm.Gamma("tau", 1, 0.003)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "cpo", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    # what is the point of this when we want cpo anyways?
    icpo = pm.Deterministic("icpo", 1 / f)
    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    trace = pm.sample(10000, target_accept=0.99)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 33 seconds.


In [9]:
icpo_df = az.summary(trace, var_names="icpo", round_to=5, filter_vars="like")

ICPO results match BUGS.

In [10]:
icpo_df

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
icpo[0],28.48666,4.75295,20.20139,37.34861,0.03951,0.02828,15107.79395,18032.10753,1.00020
icpo[1],47.91952,24.17496,22.41976,86.30776,0.17027,0.12170,21532.39322,24745.01173,1.00017
icpo[2],92.92425,51.72831,35.15217,174.70416,0.37219,0.26318,19689.56047,23110.55477,1.00016
icpo[3],40.55830,10.52069,25.27057,59.74774,0.07838,0.05574,19200.10257,22358.59371,1.00005
icpo[4],40.82864,8.73963,26.88384,57.21255,0.05018,0.03662,32769.55339,24108.99653,1.00018
icpo[5],28.33616,4.88503,20.20764,37.65274,0.04207,0.03016,14260.17643,17465.07218,1.00015
icpo[6],29.65547,5.14496,21.01779,39.50509,0.04093,0.02931,16604.65317,19681.52195,1.00021
icpo[7],28.91757,4.93592,20.49599,38.15588,0.04112,0.02941,15064.13031,17959.46568,1.00021
icpo[8],29.78635,5.54458,20.57100,40.03234,0.04704,0.03372,14747.72818,17436.35497,1.00021
icpo[9],34.93561,7.72395,22.69570,48.92397,0.05208,0.03736,23508.39543,23344.19333,1.00021


In [11]:
cpo_means = 1 / icpo_df["mean"].to_numpy()
cpo_means

array([0.03510415, 0.02086832, 0.01076145, 0.02465587, 0.02449261,
       0.0352906 , 0.03372059, 0.03458105, 0.03357242, 0.02862409,
       0.01927305, 0.03315375, 0.01076145, 0.01491499, 0.0323937 ,
       0.03518135, 0.02518993, 0.01562921, 0.00015717, 0.01909712,
       0.03518135])

## Part B

In [12]:
mlcpo = -2 * np.log(cpo_means)
MPQ = np.sum(mlcpo)

In [18]:
with pm.Model() as model_b:
    tau = pm.Gamma("tau", 1, 0.002)
    s2 = pm.Deterministic("s2", 1 / tau)
    b = pm.Normal("b", 0, tau=0.00001, shape=2)

    m = b[0] + b[1] * x
    r = y - m
    f = pm.Deterministic(
        "f", (tau / (2 * np.pi)) ** 0.5 * pm.math.exp(-0.5 * tau * r**2)
    )
    c = pm.Deterministic("c", cpo_means / f)

    g1 = pm.Deterministic("g1", c * pm.math.log(c))
    g2 = -pm.math.log(c)
    g3 = 0.5 * pm.math.abs_(c - 1)
    g4 = (c - 1) ** 2

    pm.Deterministic("s_g1", pm.math.sum(g1))
    pm.Deterministic("s_g2", pm.math.sum(g2))
    pm.Deterministic("s_g3", pm.math.sum(g3))
    pm.Deterministic("s_g4", pm.math.sum(g4))

    pm.Normal("likelihood", mu=m, tau=tau, observed=y)

    traceb = pm.sample(2000, target_accept=0.95)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [tau, b]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 13 seconds.


In [19]:
az.summary(traceb, var_names="c", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
c[0],0.999,0.170,0.714,1.319
c[1],0.990,0.481,0.491,1.757
c[2],0.999,0.532,0.388,1.881
c[3],0.999,0.254,0.627,1.479
c[4],1.000,0.209,0.663,1.375
c[5],0.998,0.175,0.701,1.322
c[6],0.999,0.177,0.719,1.339
c[7],0.999,0.174,0.716,1.334
c[8],0.999,0.189,0.685,1.337
c[9],1.000,0.224,0.664,1.403


In [20]:
az.summary(traceb, var_names="s_g", filter_vars="like", kind="stats")

,mean,sd,hdi_3%,hdi_97%
s_g1,2.647,26.319,-2.720,8.963
s_g2,2.344,1.209,0.121,3.835
s_g3,2.776,3.038,1.153,5.255
s_g4,32.141,1055.466,0.565,14.389


In [21]:
az.summary(traceb, var_names="g1", kind="stats")

,mean,sd,hdi_3%,hdi_97%
g1[0],0.013,0.181,-0.259,0.346
g1[1],0.079,0.726,-0.367,0.923
g1[2],0.110,0.791,-0.368,1.165
g1[3],0.028,0.295,-0.311,0.545
g1[4],0.021,0.233,-0.286,0.418
g1[5],0.013,0.187,-0.264,0.348
g1[6],0.015,0.190,-0.269,0.349
g1[7],0.014,0.186,-0.272,0.341
g1[8],0.016,0.206,-0.274,0.370
g1[9],0.024,0.261,-0.298,0.431


In [22]:
az.waic(traceb, scale="deviance")

/Users/aaron/mambaforge/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/stats.py:1661: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 21 observations log-likelihood matrix.

              Estimate       SE
deviance_waic   165.11    10.02
p_waic            3.27        -

There has been a warning during the calculation. Please check the results.

In [23]:
%watermark --iversions -v

Python implementation: CPython
Python version       : 3.10.4
IPython version      : 8.4.0

numpy: 1.22.4
pymc : 4.0.1
arviz: 0.12.1

